In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os

/home/ec2-user/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Loading data

In [2]:
direc=os.path.dirname(os.path.realpath('__file__'))
os.path.join(direc)
table = pd.read_csv("category_prediction.csv", names=['userId','age','sex','date','dayofweek','amount','geoId','catId','title'])
table = table.sort_values(by='date')
table['catId'].astype(int)
table['date'] = pd.to_datetime(table['date'], format='%Y-%m-%d')
table = table.set_index(['date'])

print table

                     userId  age  sex  dayofweek  amount       geoId   catId  \
date                                                                           
2018-01-01 00:00:04  537425   56    0          2    3900   서울특별시 종로구  321110   
2018-01-01 00:00:06  484684   32    1          2   10800   서울특별시 양천구  241410   
2018-01-01 00:00:17  484684   32    1          2    9500   서울특별시 양천구  241112   
2018-01-01 00:00:47  484684   32    1          2    2400   서울특별시 동작구  321110   
2018-01-01 00:01:56  545830   52    1          2    4500   서울특별시 도봉구  321110   
2018-01-01 00:02:11  534536   47    0          2   22200   서울특별시 노원구  221210   
2018-01-01 00:02:42  517178   41    0          2    8500   서울특별시 금천구  321110   
2018-01-01 00:02:56  282786   32    1          2    9370   서울특별시 관악구  321110   
2018-01-01 00:03:01  490638   57    0          2    3120  서울특별시 영등포구  321110   
2018-01-01 00:03:14  449134   36    0          2    7000   서울특별시 은평구  561110   
2018-01-01 00:05:28  494586   49    1   

# Category filtering:: 외식,유흥,문화예술,뷰티

In [3]:
#외식/유흥/문화예술/뷰티
table = table.query('(catId >= 220000 and catId <=240000) or (catId>=260000 and catId<=270000) or (catId>=560000 and catId<=570000) or (catId>=460000 and catId<= 470000)')
print table


                     userId  age  sex  dayofweek  amount       geoId   catId  \
date                                                                           
2018-01-01 00:02:11  534536   47    0          2   22200   서울특별시 노원구  221210   
2018-01-01 00:03:14  449134   36    0          2    7000   서울특별시 은평구  561110   
2018-01-01 00:09:12  501328   31    0          2   79000   서울특별시 중랑구  221111   
2018-01-01 00:09:13  491320   50    1          2   38000   서울특별시 중랑구  261310   
2018-01-01 00:09:21  143856   36    1          2    8000   서울특별시 용산구  221810   
2018-01-01 00:10:58  539205   28    1          2   25500   서울특별시 종로구  221815   
2018-01-01 00:11:32  423819   30    1          2    5500   서울특별시 강남구  261310   
2018-01-01 00:12:53  218341   23    0          2   16000   서울특별시 송파구  222113   
2018-01-01 00:13:37  489516   32    1          2   18000   서울특별시 성동구  561110   
2018-01-01 00:13:58  421620   37    0          2    8000   서울특별시 마포구  261310   
2018-01-01 00:14:24  354398   34    1   

# Input scaling

In [4]:
#min-max scaling
epsilon = 0.00000000001
print table['amount'].min()
print table['amount'].max()

#table['age'] =    ( table['age'] - table['age'].min() ) /( table['age'].max()-table['age'].min()+epsilon)
table['amount'] = (table['amount'] - table['amount'].min() ) / (table['amount'].max() - table['amount'].min() + epsilon )


1
254100000


# Hyper parameters

In [5]:
factorized_cat = pd.factorize(np.array(table['catId']))
factorized_geo = pd.factorize(np.array(table['geoId']))

original_cat={}
leng = len(factorized_cat[0])
catList = table['catId']

for i in range(0,leng):
    original_cat[ factorized_cat[0][i] ] = catList[i]
    
#params
table['catId'] = factorized_cat[0] 
catMin = int(table.agg({'catId':'min'}))
catMax = int(table.agg({'catId':'max'}))

table['geoId'] = factorized_geo[0]
geoMin = int(table.agg({'geoId':'min'}))
geoMax = int(table.agg({'geoId':'max'}))


params={}
params['catCount'] = catMax+1
params['geoCount'] = geoMax+1
params['maxLen'] = 31

params['input_size'] = 1+2+(7)+ params['catCount'] + params['geoCount']

print "Category minmax     : [%s,%s]"% (catMin,catMax)
print "Geo minmax          : [%s,%s]"% (geoMin,geoMax)
print "user Sequence       : [%s~%s]"%(0,params['maxLen']-1)



Category minmax     : [0,140]
Geo minmax          : [0,27]
user Sequence       : [0~30]


# Time window duration

In [6]:
def timerange(mode):

    if mode is 'train':
        input_s = pd.date_range(start='2018-01-01 00:00:00',periods=4,freq='10d').astype(str)
        input_e = pd.date_range(start='2018-01-31 23:59:59',periods=4,freq='10d').astype(str)
        target_s = pd.date_range(start='2018-02-01 00:00:00',periods=4,freq='10d').astype(str)
        target_e = pd.date_range(start='2018-02-10 23:59:59',periods=4,freq='10d').astype(str)
        return zip(input_s,input_e,target_s,target_e)
    
    elif mode is 'test':
        input_s = pd.date_range(start='2018-03-01 00:00:00',periods=1,freq='1d').astype(str)
        input_e = pd.date_range(start='2018-03-31 23:59:59',periods=1,freq='1d').astype(str)
        target_s = pd.date_range(start='2018-04-01 00:00:00',periods=1,freq='1d').astype(str)
        target_e = pd.date_range(start='2018-04-10 23:59:59',periods=1,freq='1d').astype(str)
        return zip(input_s,input_e,target_s,target_e)
    


# Data preprocessing with time window

In [7]:
import time
def df2seq(df,params):
    
    #dataframe to Sequence
    seq_input = []  # (batch,maxLen,input_size)
    seq_label = []  # (batch,maxLen,output_size)
    seq_mask = [] # (batch,maxLen,1)
    seq_len = []    # (batch)
    
    maxLen = params['maxLen']
    categoryCount = params['catCount']
    geoCount = params['geoCount']
     
    tp = timerange( params['mode'] )
    
    
    for s,e,ts,te in tp:

        userHistories = df.loc[s:e].groupby(['userId'])
        userFutures = df.loc[ts:te].groupby(['userId'])
        
        user_future = userFutures.groups.keys()
        user_history= userHistories.groups.keys()
           
        print s,e
        
        sss = time.time()
        for user in user_future:
        
            if user not in user_history:
                continue
            
            userData = userHistories.get_group((user))
            userTarget = userFutures.get_group((user))
            
            example = userData.loc[s:e]
            answer  = userTarget.loc[ts:te]
                        
            start_time = pd.to_datetime(s)
            end_time = pd.to_datetime(e)
            
            #user features
            sex_features = np.zeros((maxLen,2))
            age_features = np.zeros((maxLen,1))
            
            #time features
            week_features = np.zeros((maxLen,7))
            
            #transaction features
            geo_features = np.zeros((maxLen,geoCount))
            cat_features = np.zeros((maxLen,categoryCount))
            
            label = np.zeros((maxLen,categoryCount))
            
            dateList = example.index
            ageList = example['age']
            amoList = example['amount']
            sexList = example['sex']
            weekList= example['dayofweek']
            catList = example['catId']
            geoList = example['geoId']
             
            #user features
            sex_features[:,sexList[0]]=1
            age_features[:,0] = ageList[0]
            
            #time feature
            dayofweek_s = start_time.dayofweek
            for day in range(maxLen):
                week_features[day][(dayofweek_s + day)%7]=1
            
            #transaction features
            length = len(dateList)
            for t in range(length):
                today = (dateList[t]-start_time).days
                past = range( max(0, today-10),today)
                label[past , [ catList[t] ] * len(past) ]=1 
                geo_features[today][geoList[t]]=1
                cat_features[today][catList[t]]=1
                mask[today]=1
                
            temp = np.zeros((maxLen,0))
            temp = np.concatenate((temp,age_features),axis=1)
            temp = np.concatenate((temp,sex_features),axis=1)
            temp = np.concatenate((temp,week_features),axis=1)
            temp = np.concatenate((temp,geo_features),axis=1)
            temp = np.concatenate((temp,cat_features),axis=1)
            
            seq_input.append(temp)
            seq_label.append(label)
            seq_len.append(length)
            seq_mask.append(mask)
            
        print("--- %s seconds ---" %(time.time() - sss))
    return np.array(seq_input),np.array(seq_label),np.array(mask)


In [8]:
params['mode']='train'
seq_train_input, seq_train_label = df2seq(table,params)
print seq_train_input.shape,seq_train_label.shape

2018-01-01 2018-01-31 23:59:59
--- 72.2855429649 seconds ---
2018-01-11 2018-02-10 23:59:59
--- 72.9831211567 seconds ---
2018-01-21 2018-02-20 23:59:59
--- 87.1293208599 seconds ---
2018-01-31 2018-03-02 23:59:59
--- 68.6272041798 seconds ---
(70069, 31, 179) (70069, 31, 141)


In [9]:
params['mode']='test'
seq_test_input, seq_test_label= df2seq(table,params)
print seq_test_input.shape, seq_test_label.shape

2018-03-01 2018-03-31 23:59:59
--- 72.3880958557 seconds ---
(16550, 31, 179) (16550, 31, 141)


In [10]:
def converter(userHistories,s,e,userId,params):
    
    maxLen = params['maxLen']
    categoryCount = params['catCount']
    geoCount = params['geoCount']
     
    seq_input=[]
    userData = userHistories.get_group(userId)
    example = userData.loc[s:e]
    
    start_time = pd.to_datetime(s)
    end_time = pd.to_datetime(e)
            
    #user features
    sex_features = np.zeros((maxLen,2))
    age_features = np.zeros((maxLen,1))
            
    #time features
    week_features = np.zeros((maxLen,7))
            
    #transaction features
    geo_features = np.zeros((maxLen,geoCount))
    cat_features = np.zeros((maxLen,categoryCount))
            
    label = np.zeros((maxLen,categoryCount))
            
    dateList = example.index
    ageList = example['age']
    amoList = example['amount']
    sexList = example['sex']
    weekList= example['dayofweek']
    catList = example['catId']
    geoList = example['geoId']
             
    #user features
    sex_features[:,sexList[0]]=1
    age_features[:,0] = ageList[0]
            
    #time feature
    dayofweek_s = start_time.dayofweek
    for day in range(maxLen):
        week_features[day][(dayofweek_s + day)%7]=1
            
    #transaction features
    length = len(dateList)
    for t in range(length):
        today = (dateList[t]-start_time).days
        geo_features[today][geoList[t]]=1
        cat_features[today][catList[t]]=1
                            
    temp = np.zeros((maxLen,0))
    temp = np.concatenate((temp,age_features),axis=1)
    temp = np.concatenate((temp,sex_features),axis=1)
    temp = np.concatenate((temp,week_features),axis=1)
    temp = np.concatenate((temp,geo_features),axis=1)
    temp = np.concatenate((temp,cat_features),axis=1)
            
    seq_input.append(temp)
    
    return np.array(seq_input)

In [13]:
def lstmcell(size, prob_tensor):
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(size)
    lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm_cell,output_keep_prob=prob_tensor)
    
    return lstm_dropout

max_len = params['maxLen']
input_size = params['input_size']
output_size= params['catCount']

tf.reset_default_graph()

#define lstm
sequence = tf.placeholder( tf.float32, [None, max_len, input_size ])
labels = tf.placeholder( tf.float32, [None,max_len,output_size])
keep_prob = tf.placeholder(tf.float32)

dynamic_length= tf.placeholder(tf.int32, [None])


#Model
rnn_layers = [ lstmcell(size,keep_prob) for size in [50]]
multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)
outputs,_ = tf.nn.dynamic_rnn(multi_rnn_cell, sequence , dtype=tf.float32)

#predictions = outputs
predictions = (tf.contrib.layers.fully_connected(outputs, output_size))


#print predictions.shape
topk = tf.nn.top_k(predictions,k=10,sorted=True).indices
tops = tf.nn.top_k(predictions,k=10,sorted=True).values
#
loss = tf.reduce_mean(tf.square(labels-predictions))

train = tf.train.AdamOptimizer(learning_rate= 0.0001).minimize(loss)

epochs=3000
batchSize = 4000
dataSize = 70069

sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())
    
for it in range(epochs):
    
    st=time.time()
    for b in range((dataSize/batchSize)):
        batch_s = b*batchSize
        batch_e = min(dataSize,batch_s + batchSize)
        l,_ = sess.run([loss,train],feed_dict={sequence: seq_train_input[batch_s:batch_e],
                                              labels:seq_train_label[batch_s:batch_e],
                                                  keep_prob:0.5} )
    closs=0
    cnt=0;
    for b in range((16550/batchSize)):
        batch_s = b*batchSize
        batch_e = min(16550,batch_s + batchSize)
        l2= sess.run(loss,feed_dict={sequence: seq_test_input[batch_s:batch_e],
                                              labels: seq_test_label[batch_s:batch_e],
                                              keep_prob:1.0})
        cnt+=1.0
        closs+=l2;
        
    closs/=cnt;
    
    
    
    print ("Loss[%s/%s]:%s  %s"%(it,epochs,closs, time.time() -st))
    if(it%10 ==0):
        userHistories = table['2018-03-01 00:00:00' : '2018-03-31 23:59:59' ].groupby(['userId'])
        test_users =  [329439,463618, 536284,551211,453264,420850,166536,
                             434771,391746,503676,178238,567126,291711]
        
        for user in test_users:
            ex = converter(userHistories,'2018-03-01 00:00:00', '2018-03-31 23:59:59',user,params)
        
            R=sess.run(topk,feed_dict={sequence: ex,keep_prob:1.0})
            R2=sess.run(tops,feed_dict={sequence: ex,keep_prob:1.0})
            
            print R[0][-1]
            print R2[0][-1]
            print '\n'
    


Loss[0/3000]:0.029947944916784763  2.27107000351
[ 93 136  81  59  78 120  67  94  58  34]
[0.580629   0.4506958  0.43890303 0.38173404 0.3736439  0.35118663
 0.3448674  0.32135305 0.31778738 0.2965713 ]


[ 93 136  81 120  78  59  58  67  94  34]
[0.5927826  0.47899702 0.43838486 0.37138224 0.36639106 0.36057198
 0.35577083 0.33376953 0.30650476 0.2883829 ]


[ 93 136  58  59  81 120  84  34  67  78]
[0.58391476 0.5319041  0.51988536 0.41654935 0.41108698 0.38276383
 0.34962502 0.29937974 0.2949144  0.2939724 ]


[ 93  58 136  59  81 120  84 135  34  78]
[0.5944151  0.5485549  0.54512936 0.41596413 0.4032308  0.38964945
 0.36398438 0.29929084 0.29017568 0.28835696]


[ 93 136  58 120  81  59  84  34 135  67]
[0.6018033  0.5679733  0.52737397 0.4215758  0.40829596 0.39053556
 0.3339144  0.31511414 0.30460554 0.30391836]


[ 93  58 136  59  81 120  84 135  34 138]
[0.5992735  0.5740126  0.558519   0.43056804 0.41661993 0.39298698
 0.38760006 0.3126803  0.2885855  0.2848375 ]


[ 93 136 

[60 18  0  1  2  3  4  5  6  7]
[0.03162606 0.01812313 0.         0.         0.         0.
 0.         0.         0.         0.        ]


[60 18 97  0  1  2  3  4  5  6]
[0.0278304  0.02300688 0.00870068 0.         0.         0.
 0.         0.         0.         0.        ]


[60  0  1  2  3  4  5  6  7  8]
[0.02051613 0.00745668 0.         0.         0.         0.
 0.         0.         0.         0.        ]


[60  0  1  2  3  4  5  6  7  8]
[0.0118457  0.00943352 0.         0.         0.         0.
 0.         0.         0.         0.        ]


[0 1 2 3 4 5 6 7 8 9]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


Loss[31/3000]:0.006319464650005102  2.19864010811
Loss[32/3000]:0.006303485133685172  2.22190904617
Loss[33/3000]:0.006286986637860537  2.22210788727
Loss[34/3000]:0.006270372192375362  2.22047591209
Loss[35/3000]:0.006254844949580729  2.21008086205
Loss[36/3000]:0.006241428665816784  2.20500993729
Loss[37/3000]:0.006230309489183128  2.17781305313
Loss[38/3000]:0.006220589159056544  2.

[ 0 17  8 23 11  1  2  3  4  5]
[0.11952798 0.07137457 0.04934791 0.04365604 0.03652011 0.03299628
 0.         0.         0.         0.        ]


[ 0 17 23 11  1  8  2  3  4  5]
[0.10975415 0.04772409 0.04444119 0.03959493 0.03856243 0.03404638
 0.         0.         0.         0.        ]


[ 0 17 23  1  8 11  2  3  4  5]
[0.13236815 0.08174732 0.05030428 0.03075754 0.02805049 0.00451929
 0.         0.         0.         0.        ]


[ 0 17 23  1  8 11  2  3  4  5]
[0.12290159 0.06378783 0.05002353 0.02536657 0.01645784 0.00317971
 0.         0.         0.         0.        ]


[ 0 17 20  1 11  2  3  4  5  6]
[0.13113473 0.10142387 0.0876873  0.04714572 0.00126165 0.
 0.         0.         0.         0.        ]


Loss[71/3000]:0.006135395262390375  2.16567897797
Loss[72/3000]:0.006133433082140982  2.18648886681
Loss[73/3000]:0.0061314781196415424  2.19421815872
Loss[74/3000]:0.006129510700702667  2.19436812401
Loss[75/3000]:0.00612745329272002  2.19186878204
Loss[76/3000]:0.0061255

Loss[106/3000]:0.0060780157800763845  2.16424798965
Loss[107/3000]:0.006076773861423135  2.16620707512
Loss[108/3000]:0.006075402954593301  2.1618270874
Loss[109/3000]:0.006074103061109781  2.16391301155
Loss[110/3000]:0.006072891061194241  2.16409897804
[ 0 23  8 17 11  1 20 34  2  3]
[0.13057828 0.08004021 0.07748473 0.07698259 0.0545535  0.05250936
 0.02844814 0.01510947 0.         0.        ]


[ 0 23  1 17  8 11 20 40 34  2]
[0.13086687 0.08840856 0.08265226 0.07896008 0.07850324 0.04307194
 0.02628635 0.0149629  0.01184691 0.        ]


[ 0 17 20 23 11  8  1 39  2  3]
[0.14115658 0.09533659 0.05776097 0.05125131 0.04225159 0.01707713
 0.01295439 0.00061844 0.         0.        ]


[ 0 20 17 23  1 11  8 21 40 34]
[0.13633105 0.0840573  0.08239885 0.06878471 0.04058389 0.03751286
 0.00609225 0.00272234 0.00231341 0.00078164]


[17  0 20 11 23  8 27 34  1 60]
[0.14416662 0.13964008 0.07724074 0.05167277 0.05104693 0.02796256
 0.0176816  0.00495539 0.00391638 0.00289712]


[ 0 17 20 

[ 0 17  8 23  1 11 34 20 40  2]
[0.14947751 0.11582188 0.09615906 0.06462564 0.04197951 0.03685714
 0.02686682 0.02039741 0.01270437 0.        ]


[ 0  8 23 11  1 17 40  2  3  4]
[0.07527276 0.05835736 0.04099652 0.03075067 0.02608572 0.0169338
 0.01370634 0.         0.         0.        ]


[17  0  8 23 20  1 11 34 36  2]
[0.17852499 0.14977472 0.08896555 0.06187072 0.03967459 0.03447323
 0.03240704 0.02871997 0.02182095 0.        ]


[17  0  8 23 11 20  1 36 34 60]
[0.10339637 0.07916961 0.05799471 0.03943716 0.03169132 0.02353115
 0.02164697 0.00830171 0.00698505 0.00317099]


[ 0 20 17 23  1 11 40 21 34  8]
[0.11290494 0.07637864 0.06573807 0.0578839  0.05002541 0.03680842
 0.02922594 0.01398748 0.00701223 0.00429043]


Loss[141/3000]:0.006040773703716695  2.18815493584
Loss[142/3000]:0.006039892672561109  2.18723583221
Loss[143/3000]:0.006038965890184045  2.18575906754
Loss[144/3000]:0.006038236431777477  2.18373918533
Loss[145/3000]:0.006037379498593509  2.18684387207
Loss[146/30

...

[17 42  0  1  2  3  4  5  6  7]
[0.009432   0.00479323 0.         0.         0.         0.
 0.         0.         0.         0.        ]


[42 40  6  0  1  2  3  4  5  7]
[0.00957163 0.00350464 0.00029002 0.         0.         0.
 0.         0.         0.         0.        ]


[0 1 2 3 4 5 6 7 8 9]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


[21 17  0  1  2  3  4  5  6  7]
[0.01130662 0.00119208 0.         0.         0.         0.
 0.         0.         0.         0.        ]


[ 1  9 40  3  0  2  4  5  6  7]
[0.00931799 0.00565076 0.00400782 0.00244608 0.         0.
 0.         0.         0.         0.        ]


Loss[1211/3000]:0.0058822608552873135  2.16834115982
Loss[1212/3000]:0.005882644327357411  2.17544102669
Loss[1213/3000]:0.005882459809072316  2.16965889931
Loss[1214/3000]:0.005882206605747342  2.17148804665
Loss[1215/3000]:0.005882174242287874  2.16551589966
Loss[1216/3000]:0.005883058067411184  2.16428899765
Loss[1217/3000]:0.005883089732378721  2.16234493256
Loss[1218/3000]:0.00588

[17 42 18  0  1  2  3  4  5  6]
[0.01224644 0.00451936 0.00139875 0.         0.         0.
 0.         0.         0.         0.        ]


[42 40  6  0  1  2  3  4  5  7]
[0.00912006 0.00369757 0.00032702 0.         0.         0.
 0.         0.         0.         0.        ]


[0 1 2 3 4 5 6 7 8 9]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


[21 17  0  1  2  3  4  5  6  7]
[0.01198591 0.00420998 0.         0.         0.         0.
 0.         0.         0.         0.        ]


[ 1  9 40  3 20  0  2  4  5  6]
[0.00749923 0.00664905 0.00453122 0.00277679 0.00248126 0.
 0.         0.         0.         0.        ]


Loss[1251/3000]:0.00588218797929585  2.1613740921
Loss[1252/3000]:0.005882539786398411  2.16538906097
Loss[1253/3000]:0.0058820967096835375  2.16225504875
Loss[1254/3000]:0.005881816963665187  2.18426895142
Loss[1255/3000]:0.00588236260227859  2.19570422173
Loss[1256/3000]:0.005882015568204224  2.16605901718
Loss[1257/3000]:0.005882367957383394  2.16870903969
Loss[1258/3000]:0.00588264

[17 42 18  0  1  2  3  4  5  6]
[0.02021429 0.00748654 0.00362589 0.         0.         0.
 0.         0.         0.         0.        ]


[42 40  6  9  3  8  1  0  2  4]
[0.01100515 0.0045715  0.00102267 0.00084643 0.00035629 0.00021659
 0.00019    0.         0.         0.        ]


[17  0  1  2  3  4  5  6  7  8]
[0.00917699 0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]


[21 17  0  1  2  3  4  5  6  7]
[0.01288566 0.01086216 0.         0.         0.         0.
 0.         0.         0.         0.        ]


[ 1  9 40  3 20 72  0  2  4  5]
[1.1165235e-02 9.2531303e-03 6.2719975e-03 3.9438596e-03 1.6077086e-03
 3.6160462e-05 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]


Loss[1291/3000]:0.00588573154527694  2.17143201828
Loss[1292/3000]:0.005885910126380622  2.16908121109
Loss[1293/3000]:0.005886514205485582  2.16465997696
Loss[1294/3000]:0.005886851577088237  2.16334486008
Loss[1295/3000]:0.0058874867390841246  2.16398692131

[0 1 2 3 4 5 6 7 8 9]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


[42 40  0  1  2  3  4  5  6  7]
[0.00464237 0.00208788 0.         0.         0.         0.
 0.         0.         0.         0.        ]


[17  0  1  2  3  4  5  6  7  8]
[0.0119547 0.        0.        0.        0.        0.        0.
 0.        0.        0.       ]


[21  0  1  2  3  4  5  6  7  8]
[0.00235466 0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]


[ 9 40  0  1  2  3  4  5  6  7]
[0.00292049 0.00064407 0.         0.         0.         0.
 0.         0.         0.         0.        ]


Loss[1331/3000]:0.005902148084715009  2.16556096077
Loss[1332/3000]:0.005901883821934462  2.16086196899
Loss[1333/3000]:0.005901942844502628  2.16708302498
Loss[1334/3000]:0.005901784170418978  2.18458509445
Loss[1335/3000]:0.005901385564357042  2.18184399605
Loss[1336/3000]:0.00590139499399811  2.17879486084
Loss[1337/3000]:0.005901362746953964  2.17962002754
Loss[1338/3000]:0.0059009266551584

KeyboardInterrupt: 


- 0: 패스트푸드/일반
- 1: 영화
- 2: 외식/회
- 9: 맥주
- 42: 화장품
- 21: 한식뷔폐
- 17: 한식/일반
- 8 :
- 31: 일식/일반
- 23: 분식/일반
- 50: 일식/돈까스
- 40: 헤어샵

In [ ]:
original_cat


In [ ]:
test_users = [329439,463618, 536284,551211,453264,420850,166536,
                             434771,391746,503676,178238,567126,291711]

#17  8 23  0 20 31  1 40 21 11
#0 23  8  1 11 50 31 34  4 28 

table['2018-03-20':'2018-04-03'].groupby(['age']).get_group(27)
    

table.groupby(['userId']).get_group(291711)

# User Sampling


- Sampling based on user group
    - 20~25 여성 = [341770, 329439, 198995, 161617, 310756, 150409, 137363, 463618, 150409, 422341]
        
    - 20~25 남성 = [3914, 481844, 396370, 103036, 481245, 459350,272707]
        
    - 26~30 여성 = [400295, 326175, 536874, 505684, 393859, 359757, 291616, 182774, 207415,273544,420960,185077]
        
    - 26~30 남성 = [550647,	329514,343291,282759,374170,254809,515139,372982,522194,556329,441214, 503106]
        
 
 